In [1]:
print("all ok")

all ok


### Training and Validate a model performance

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [4]:
df=pd.read_csv(r'../Datasets/titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df=df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

### use simpleimputer for handling missing values

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=47)

In [13]:
age_imputer=SimpleImputer()
embarked_imputer=SimpleImputer(strategy='most_frequent')

x_train_age=age_imputer.fit_transform(x_train[['Age']])
x_train_embarked=embarked_imputer.fit_transform(x_train[['Embarked']])

x_test_age=age_imputer.transform(x_test[['Age']])
x_test_embarked=embarked_imputer.transform(x_test[['Embarked']])

In [18]:
ohe_sex=OneHotEncoder(drop='first',dtype=int,sparse_output=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(drop='first',dtype=int,sparse_output=False,handle_unknown='ignore')


x_train_sex=ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked=ohe_embarked.fit_transform(x_train_embarked)

x_test_sex=ohe_sex.transform(x_test[['Sex']])
x_test_embarked=ohe_embarked.transform(x_test_embarked)

In [19]:
x_train_rem=x_train.drop(columns=['Sex','Age','Embarked'])
x_test_rem=x_test.drop(columns=['Sex','Age','Embarked'])

In [21]:
x_train_rem.head()

,Pclass,SibSp,Parch,Fare
206,3,1,0,15.8500
146,3,0,0,7.7958
211,2,0,0,21.0000
844,3,0,0,8.6625
405,2,1,0,21.0000


In [23]:
x_train_final=np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_final=np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [24]:
x_train_final.shape

(712, 8)

In [25]:
x_test_final.shape

(179, 8)

In [28]:
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()

In [30]:
clf.fit(x_train_final,y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [31]:
y_pred=clf.predict(x_test_final)

In [32]:
y_pred

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1])

In [33]:
accuracy_score(y_test,y_pred)

0.7430167597765364

In [34]:
import pickle

In [37]:
pickle.dump(ohe_sex,open(r'../Pipeline_examples/models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open(r'../Pipeline_examples/models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open(r'../Pipeline_examples/models/clf.pkl','wb'))

### Test with deployment the model

In [38]:
import pickle
import numpy as np

In [41]:
clf_loaded=pickle.load(open(r'../Pipeline_examples/models/clf.pkl','rb'))
ohe_embarked_loaded=pickle.load(open(r'../Pipeline_examples/models/ohe_embarked.pkl','rb'))
ohe_sex_loaded=pickle.load(open(r'../Pipeline_examples/models/ohe_sex.pkl','rb'))

In [42]:
test_input=np.array([3,'female',26.0,0,0,7.9250,'S'],dtype=object).reshape(1,7)

In [43]:
test_input

array([[3, 'female', 26.0, 0, 0, 7.925, 'S']], dtype=object)

In [44]:
test_input_sex=ohe_sex_loaded.transform(test_input[:,1].reshape(1,1))

c:\Users\Hp\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [45]:
test_input_sex

array([[0]])

In [46]:
test_input_embarked=ohe_embarked_loaded.transform(test_input[:,-1].reshape(1,1))


In [47]:
test_input_embarked

array([[0, 1]])

In [48]:
test_input_age=test_input[:,2].reshape(1,1)

In [49]:
test_input_age

array([[26.0]], dtype=object)

In [51]:
test_input_final=np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [52]:
test_input_final.shape

(1, 8)

In [53]:
test_input_final

array([[3, 0, 0, 7.925, 26.0, 0, 0, 1]], dtype=object)

In [54]:
test_predict=clf_loaded.predict(test_input_final)

In [55]:
test_predict

array([1])